# 📝 Lab #4 (응용): 신용카드 사기 탐지 (Credit Card Fraud Detection)

지금까지 배운 이상 탐지 기법을 활용하여 실제 금융 데이터셋에서 사기 거래를 탐지하는 현실적인 문제에 도전해 보겠습니다.

이 데이터셋은 실제 신용카드 거래 내역을 바탕으로 하며, 개인정보 보호를 위해 대부분의 특성이 비식별화(V1~V28)되어 있습니다. 

`Class` 열이 `0`이면 정상 거래, `1`이면 사기 거래를 의미합니다. 전체 약 28만 건의 거래 중 사기 거래는 약 0.17%에 불과한, 매우 **불균형한(imbalanced)** 데이터셋입니다.

### 과제 목표
1. 레이블(`Class`) 정보를 사용하지 않고 **비지도 학습 방식**으로 이상 탐지 모델(Isolation Forest 또는 One-Class SVM)을 학습시킵니다.
2. 학습된 모델을 사용해 테스트 데이터의 각 거래가 사기인지 아닌지를 예측합니다.
3. 숨겨져 있던 실제 레이블과 비교하여 모델의 성능을 **정밀도(Precision), 재현율(Recall), F1-Score, ROC-AUC** 등 다양한 지표로 평가합니다.
4. 비지도 이상 탐지의 현실적인 성능과 한계를 이해하고, 결과에 대해 분석합니다.

### 3.1 데이터 준비 (Data Preparation)

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import plotly.express as px
import plotly.graph_objects as go

# 데이터 불러오기
path = '../../datasets/ml/creditcardfraud/creditcard.csv'
df = pd.read_csv(path)

**[문제 1]** 데이터의 기본 정보를 확인하세요. 
- `df.info()`를 통해 데이터 타입과 결측치를 확인하세요.
- `df['Class'].value_counts()`를 통해 정상 거래와 사기 거래의 비율을 확인하세요.

In [ ]:
# 여기에 코드 작성
if df is not None:
    # df.info()
    # df['Class'].value_counts()

**[문제 2]** 데이터를 전처리하세요.
- `Amount`와 `Time` 컬럼은 다른 V1~V28 컬럼과 값의 범위(scale)가 크게 다릅니다. `StandardScaler`를 사용하여 이 두 컬럼을 정규화하세요.
- `Time` 컬럼은 이 분석에서 중요하지 않다고 판단하여 삭제하거나, 정규화하여 그대로 사용할 수 있습니다. 여기서는 정규화하여 사용하겠습니다.

In [ ]:
if df is not None:
    # StandardScaler 객체 생성
    scaler = StandardScaler()

    # Amount, Time 컬럼 스케일링
    df['scaled_Amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
    df['scaled_Time'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))

    # 기존 컬럼 삭제
    df = df.drop(['Time', 'Amount'], axis=1)

**[문제 3]** 데이터를 훈련 세트와 테스트 세트로 분할하세요.
- 특성 데이터 `X`와 레이블 데이터 `y`를 분리합니다. (`y`는 나중에 모델 평가에만 사용됩니다.)
- `train_test_split`을 사용하여 훈련:테스트 비율을 7:3으로 나누세요. `random_state=42`로 고정합니다.

In [ ]:
if df is not None:
    # 특성(X)과 타겟(y) 분리
    X = df.drop('Class', axis=1)
    y = df['Class']

    # 훈련/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### 3.2 모델 설계 및 학습 (Model Design & Training)

**[문제 4]** **Isolation Forest** 모델을 학습시키세요.
- `contamination` 파라미터를 실제 사기 거래 비율(약 0.0017)과 유사하게 설정합니다. 
- `n_estimators=100`, `random_state=42`로 설정합니다.
- Isolation Forest는 이상치가 포함된 전체 데이터로 학습할 수 있습니다.

In [ ]:
if df is not None:
    # Isolation Forest 모델 초기화 및 학습
    # if_model = ?
    # if_model.fit(?)

### 3.3 이상치 탐지 및 평가 (Detection & Evaluation)

**[문제 5]** 학습된 모델을 사용하여 테스트 세트의 이상치를 예측하고, 실제 레이블과 비교하여 성능을 평가하세요.
- `X_test` 데이터에 대해 예측을 수행합니다. (정상: 1, 이상: -1)
- `sklearn`의 평가 지표는 보통 0(정상), 1(이상)을 기준으로 하므로, 모델의 예측값(`-1`)을 실제 레이블의 이상치(`1`)와 매핑하는 작업이 필요합니다. (`y_pred[y_pred == 1] = 0`, `y_pred[y_pred == -1] = 1`)
- `confusion_matrix`와 `classification_report`를 출력하여 정밀도, 재현율 등을 확인하세요. [cite: 108, 112]

In [ ]:
if df is not None:
    # 테스트 데이터에 대한 예측
    # y_pred_if = ?

    # 레이블 변환 (정상 1 -> 0, 이상 -1 -> 1)
    # y_pred_if[y_pred_if == 1] = 0
    # y_pred_if[y_pred_if == -1] = 1

    # 성능 평가
    # print("--- Isolation Forest 평가 결과 ---")
    # print(confusion_matrix(?, ?))
    # print(classification_report(?, ?))

**[문제 6]** 모델의 **ROC-AUC 점수**를 계산하고, **ROC 곡선**을 시각화하세요.
- ROC 곡선은 모델의 `decision_function`으로 얻은 점수를 사용해야 합니다. (점수가 낮을수록 이상치)
- 사이킷런의 `roc_auc_score`와 `roc_curve` 함수를 사용하세요.
- Plotly를 사용하여 ROC 곡선을 그리세요.

In [ ]:
if df is not None:
    # 이상치 점수 계산 (값이 작을수록 이상치이므로 부호를 바꿈)
    # scores_if = -if_model.decision_function(?)

    # ROC-AUC 점수 계산
    # auc_score = roc_auc_score(?, ?)
    # print(f"Isolation Forest ROC-AUC: {auc_score:.4f}")

    # ROC 곡선 데이터 생성
    # fpr, tpr, thresholds = roc_curve(?, ?)

    # ROC 곡선 시각화
    # fig = go.Figure()
    # fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Isolation Forest (AUC={auc_score:.4f})'))
    # fig.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)
    # fig.update_layout(title='ROC Curve',
    #                   xaxis_title='False Positive Rate',
    #                   yaxis_title='True Positive Rate')
    # fig.show()

### 3.4 결과 해석 및 개선 방안 (Analysis & Next Steps)

**[문제 7]** 위 평가 결과를 바탕으로 다음 질문에 답해보세요.

1.  이 모델의 사기 탐지 **재현율(Recall)**은 얼마인가요? 이 수치가 사기 탐지에서 왜 중요한가요? 
2.  이 모델의 **정밀도(Precision)**는 얼마인가요? 재현율과 비교했을 때 어떤 특징을 보이며, 이는 무엇을 의미하나요? 
3.  ROC-AUC 점수는 어느 정도로 나왔나요? 이 점수를 어떻게 해석할 수 있나요?
4.  만약 이 모델의 성능을 더 개선하고 싶다면 어떤 방법을 시도해볼 수 있을까요? (예: 하이퍼파라미터 튜닝, 다른 알고리즘 사용, 지도학습과의 비교 등)